# Step16 Complex Computation Graph(Implementation)

## 16.1 Add Generation

In [20]:
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation=0 #number of generations

    def set_creator(self, func): #everytime the set_creator function is called, +1 to the num. of gen.
        self.creator = func
        self.generation= func.generation+1 #parent_generation+1

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx 
            
                if x.creator is not None:
                    funcs.append(x.creator)
    def cleargrad(self):
        self.grad=None

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [21]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation=max([x.generation for x in inputs]) 
        #among the inputs, choose one that has the largest value of gen. 
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

## 16.2 In the order of Generation

In [22]:
generations=[2,0,1,6,2]
funcs=[]

for g in generations:
    f=Function()
    f.generation=g
    funcs.append(f)
[f.generation for f in funcs]

[2, 0, 1, 6, 2]

In [23]:
funcs.sort(key=lambda x: x.generation)
[f.generation for f in funcs]

[0, 1, 2, 2, 6]

In [24]:
f=funcs.pop()
f.generation

6

## 16.3 Backward function of Variable Class

In [25]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation=0 #number of generations

    def set_creator(self, func): #everytime the set_creator function is called, +1 to the num. of gen.
        self.creator = func
        self.generation= func.generation+1 #parent_generation+1

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set=set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx 
            
                if x.creator is not None:
                    add_func(x.creator)
    def cleargrad(self):
        self.grad=None

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
%run Codes/step16.ipynb

In [9]:
import numpy as np

x=Variable(np.array(2.0))
a=square(x)
y=add(square(a),square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0
